In [1]:
import os
os.chdir('..')

In [2]:
import keplergl
import pickle
import warnings
import yaml

from src.data import data_utils
from src.geo import geo_utils
from src.geo.geo_object import GeoObject, FeaturizedLayer
from src.features.featurize import Featurization, GeoFeaturizer

warnings.filterwarnings('ignore')
CONFIG_PATH = 'config/config_moscow.yaml'

### Read a config file

In [3]:
config = yaml.load(open(CONFIG_PATH), Loader=yaml.FullLoader)
config

{'project_name': 'Moscow',
 'osm_shp_url': 'http://download.geofabrik.de/russia/central-fed-district-latest-free.shp.zip',
 'force_download_data': False,
 'path_to_data_raw': 'data/raw',
 'path_to_data_interim': 'data/interim',
 'path_to_data_processed': 'data/processed',
 'objects_to_featurize': {'h3': {'region_of_interest_shp': 'data/raw/Moscow.shp',
   'hexagons_resolution': 9},
  'polygons': {'layer_shp': 'coverage_moscow.shp'},
  'points': {'layer_shp': 'places.shp'}},
 'geo_objects': [{'eat_point': {'shp': 'gis_osm_pois_free_1.shp',
    'filters': {'fclass': ['restaurant',
      'cafe',
      'biergarten',
      'fast_food',
      'food_court']}}},
  {'drink_point': {'shp': 'gis_osm_pois_free_1.shp',
    'filters': {'fclass': ['pub', 'bar']}}},
  {'shop_point': {'shp': 'gis_osm_pois_free_1.shp',
    'filters': {'fclass': ['bakery',
      'books',
      'convenience',
      'department_store',
      'mall',
      'supermarket',
      'beverages',
      'butcher',
      'chemist',


### Download OSM dump

In [4]:
# data_utils.download_osm_data(config['osm_shp_url'], config['path_to_data_raw'] + config['project_name'])

### Create some objects from the config file

In [5]:
region_of_interest = geo_utils.load_shp(config['objects_to_featurize']['h3']['region_of_interest_shp'])

In [6]:
feat_layer_hex = FeaturizedLayer.from_hexagons(
    config['objects_to_featurize']['h3']['region_of_interest_shp'], 
    config['objects_to_featurize']['h3']['hexagons_resolution'])
feat_layer_hex.layer.plot();

In [7]:
# feat_layer_shp = FeaturizedLayer.from_shp(
#     config['path_to_data_raw'] + '/' + config['objects_to_featurize']['polygons']['layer_shp'])
# feat_layer_shp.layer.plot();

In [8]:
geo_objects = {}
for geo_obj_desc in config['geo_objects']:
    geo_object_name = list(geo_obj_desc.keys())[0]
    shp_filename = geo_obj_desc[geo_object_name]['shp']
    filters = geo_obj_desc[geo_object_name]['filters']
    geo_object = GeoObject(geo_object_name, config['path_to_data_raw'] 
                           + '/' + config['project_name'] + '/' + shp_filename, 
                           filters, region_of_interest)
    geo_objects[geo_object_name] = geo_object

In [9]:
featurizations = []

for feat in config['featurizations']:
    name = list(feat.keys())[0]
    operation = feat[name]['operation']
    geo_object_name = feat[name]['geo_object']
    featurization = Featurization(name, operation, geo_object_name, feat[name])
    featurizations.append(featurization)

### Create the main object - Featurizer and start geo featurization

In [10]:
featurizer = GeoFeaturizer(feat_layer_hex, geo_objects, featurizations)
# featurizer = GeoFeaturizer(feat_layer_shp, geo_objects, featurizations)
featurizer.featurize()

In [20]:
# featurizer.featurized_layer.layer.to_file("tmp.shp")

In [26]:
featurizer.featurized_layer.layer.head()

,hex_id,geometry,eat_number,drink_number,shop_number,education_number,mobile_phone_shop_number,railway_station_point_number,bus_stop_point_number,building_number,dist_distance_to_nearest_railway_station,dist_distance_to_nearest_bus_stop
0,8911aa09307ffff,"POLYGON ((37.29423 55.80143, 37.29663 55.80038...",2,0,0,0,0,0,0,3,0.078450,0.002605
1,8911aa0930fffff,"POLYGON ((37.29627 55.79856, 37.29867 55.79752...",0,0,0,0,0,0,1,3,0.078421,0.001426
2,8911aa09303ffff,"POLYGON ((37.29113 55.79884, 37.29352 55.79780...",0,0,0,0,0,0,0,0,0.082465,0.005395
3,8911aa0933bffff,"POLYGON ((37.29938 55.80115, 37.30177 55.80010...",2,0,0,0,0,0,0,2,0.074392,0.003192
4,8911aa78a8bffff,"POLYGON ((37.68358 55.71897, 37.68597 55.71792...",0,0,0,0,0,0,0,17,0.005551,0.003080


### Visualize results using Kepler

In [3]:
# import keplergl

# with open('map_config_railway', 'rb') as f:
#     config1 = pickle.load(f)

# map_kepler = keplergl.KeplerGl(height=800, config=config1)
# map_kepler.add_data(data=featurizer.featurized_layer.layer, name='feat_layer')
# map_kepler

## Visualization samples

#### Number of buildings in a hexagon 3D

<img src='../samples/moscow_buildings_num_3d.png'>

#### Number of buildings in a hexagon 3D

<img src='../samples/moscow_buildins_num_water.png'>

#### Distance from a hexagon's centroid to the nearest railway/subway station

<img src='../samples/moscow_distance_to_railway.png'>

## HTML Sample

<a href='../samples/map_dist_to_railway.html'>Distance from a hexagon's centroid to the nearest railway/subway station</a>

In [8]:
# operations_todo = {'distance_to_nearest', 'features_agg_buffer', 'features_of_nearest', 
#                    'count_contains', 'count_intersects', 'count_in_buffer', 
#                    'coverage_percent', 'is_in', 'count_contains_bin', 'distance_to_nearest_bin'}